In [1]:
#Cài đặt các thư viện
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Khởi tạo driver
driver = webdriver.Chrome()
driver.set_script_timeout(1000)
# Truy cập vào trang web
driver.get('https://cafef.vn/thi-truong-chung-khoan.chn')
# Đưa nguồn vào BeautifulSoup và phân tích cú pháp
soup = BeautifulSoup(driver.page_source)

In [ ]:
# Khởi tạo thời gian bắt đầu
start_time = time.time()
# Thực hiện vòng lặp và trích xuất dữ liệu
for i in range(1, 1200):
    if i % 50 == 0:
        end_time = time.time()
        print(f"Crawl item from {i-50+1} to {i} in {end_time-start_time} seconds")
        time.sleep(0.5)
    # Bấm vào nút xem thêm          
    driver.execute_script("document.getElementsByClassName('btn-viewmore')[0].click()") 
# Lấy các phần tử HTML
l = BeautifulSoup(driver.page_source).find_all('div', attrs={'class':'tlitem'})

Crawl item from 1 to 50 in 2.211787462234497 seconds
Crawl item from 51 to 100 in 5.303850889205933 seconds
Crawl item from 101 to 150 in 12.905007362365723 seconds
Crawl item from 151 to 200 in 21.639371156692505 seconds
Crawl item from 201 to 250 in 27.181602239608765 seconds
Crawl item from 251 to 300 in 32.4908721446991 seconds
Crawl item from 301 to 350 in 40.603986501693726 seconds
Crawl item from 351 to 400 in 57.14358377456665 seconds
Crawl item from 401 to 450 in 93.52760887145996 seconds
Crawl item from 451 to 500 in 138.02705001831055 seconds
Crawl item from 501 to 550 in 210.5906913280487 seconds
Crawl item from 551 to 600 in 293.17867493629456 seconds
Crawl item from 601 to 650 in 401.0387327671051 seconds
Crawl item from 651 to 700 in 543.2365446090698 seconds
Crawl item from 701 to 750 in 729.854008436203 seconds
Crawl item from 751 to 800 in 951.4647357463837 seconds
Crawl item from 801 to 850 in 1214.7552978992462 seconds
Crawl item from 851 to 900 in 1532.937417745590

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
# Trích xuất thông tin từ các phần tử HTML và lưu vào danh sách
res = []
tim_mod=[]
for i, article in enumerate(l):   
    title = article.find('h3').find('a').text
    tim = article.find('span', class_= 'time-ago')['title']
    link = article.find('h3').find('a')['href']
    res.append([tim, title, link])
   

NameError: name 'l' is not defined

In [5]:
#Chuyển đổi danh sách thành DataFrame
df = pd.DataFrame(res, columns=['Time', 'Headline','Link'])
# df.to_csv('cafef_news.csv', index=False)  

In [6]:
# df = pd.read_csv('cafef_news.csv')
df['Link'] = 'https://cafef.vn' + df['Link']+''

In [7]:
# Giả sử df đã được tạo và có cột 'Link'
df['content'] = ""
df['subtitle'] = ""
df['Keyword'] = ""

# Hàm xử lý từng URL
def fetch_content(index, url):
    try:
        response = requests.get(url, timeout=10)  # Đặt timeout tránh treo request

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # Trích xuất dữ liệu từ trang
            detail_content = soup.find("div", class_="contentdetail")
            sub_title = soup.find("h2", class_="sapo")
            keywords = soup.find('div', class_='row2')

            # Lấy danh sách từ khóa
            keyword_list = [kw.get_text(strip=True) for kw in keywords.find_all('a')] if keywords else []

            # Trả về kết quả
            return index, (
                sub_title.get_text(strip=True) if sub_title else "Không có tiêu đề",
                detail_content.get_text(strip=True) if detail_content else "Không có thông tin",
                ', '.join(keyword_list)
            )
        else:
            print(f"Không thể truy cập {url}: Mã trạng thái {response.status_code}")
            return index, ("Lỗi truy cập", "Lỗi truy cập", "")

    except Exception as e:
        print(f"Lỗi khi xử lý {url}: {e}")
        return index, ("Lỗi xử lý", "Lỗi xử lý", "")

# Sử dụng ThreadPoolExecutor để chạy đa luồng
num_threads = 10  # Số luồng song song
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    future_to_index = {executor.submit(fetch_content, index, row['Link']): index for index, row in df.iterrows()}
    
    for future in as_completed(future_to_index):
        index, (subtitle, content, keywords) = future.result()
        df.at[index, 'subtitle'] = subtitle
        df.at[index, 'content'] = content
        df.at[index, 'Keyword'] = keywords

print("Hoàn thành xử lý dữ liệu!")

Hoàn thành xử lý dữ liệu!


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Time      223 non-null    object
 1   Headline  223 non-null    object
 2   Link      223 non-null    object
 3   content   223 non-null    object
 4   subtitle  223 non-null    object
 5   Keyword   223 non-null    object
dtypes: object(6)
memory usage: 10.6+ KB


In [12]:
df.to_csv('cafef_news_details.csv', index=False)

In [ ]:
# df.to_csv('cafef_news_test1.csv', index=False)

In [14]:
df1 = pd.read_csv('cafef_news_test1.csv')

In [ ]:
import pandas as pd

df_all = pd.concat([df, df1], ignore_index=True)


In [17]:
df_all.drop_duplicates(subset=['Link'], keep='first', inplace=True)

In [18]:
df_all.to_csv('cafef_news_test.csv', index=False)

Crawl tin đồn của PLX, PetroVietNam

In [19]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Giả lập headers để tránh bị chặn
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}

# Hàm để lấy thông tin chi tiết từ bài viết
def crawl_detail_page(url):
    if not url.startswith('http'):
        url = 'https://cafef.vn' + url

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Lấy nội dung bài viết
        detail = soup.find('div', class_='detail-content')
        detail_text = detail.get_text(strip=True) if detail else 'N/A'

        # Lấy thời gian đăng bài
        time_tag = soup.find('span', class_='pdate')
        time_text = time_tag.get_text(strip=True) if time_tag else 'N/A'

        # Lấy chuyên mục
        category_tag = soup.find('a', class_='category-page__name cat')
        category_text = category_tag.get_text(strip=True) if category_tag else 'N/A'

        time.sleep(1)  # nghỉ giữa các request để tránh bị phát hiện
        return detail_text, time_text, category_text

    except requests.exceptions.RequestException as e:
        print(f"[Lỗi khi truy cập trang chi tiết]: {url} - {e}")
        return 'N/A', 'N/A', 'N/A'


# Hàm xử lý từng trang kết quả tìm kiếm
def crawl_page(url, writer):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        items = soup.find_all('div', class_='item')
        if not items:
            print(f"[Không có bài viết tại]: {url}")
            return False

        for item in items:
            try:
                title_tag = item.find('a', class_='box-category-link-title')
                title = title_tag.get_text(strip=True)
                link = title_tag['href']
                if not link.startswith('http'):
                    link = 'https://cafef.vn' + link

                desc_tag = item.find('p', class_='sapo')
                description = desc_tag.get_text(strip=True) if desc_tag else 'N/A'

                # Crawl thông tin chi tiết
                detail_content, time_posted, category = crawl_detail_page(link)

                writer.writerow([title, link, description, time_posted, category, detail_content])
                print(f"[✓] Đã lưu bài: {title}")
                time.sleep(1)  # nghỉ giữa các bài viết
            except Exception as e:
                print(f"[Lỗi xử lý bài viết]: {e}")

        return True

    except requests.exceptions.RequestException as e:
        print(f"[Lỗi khi truy cập trang kết quả]: {url} - {e}")
        return False


# Từ khóa cần tìm kiếm
keywords = ['petrolimex', 'plx', 'petrovietnam']

# Mở file CSV để lưu kết quả
with open('data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Link', 'Description', 'Time', 'Category', 'Detail Content'])

    for keyword in keywords:
        print(f"\n=== BẮT ĐẦU TÌM KIẾM: {keyword} ===")
        page_number = 1

        while True:
            if page_number == 1:
                url = f'https://cafef.vn/tim-kiem.chn?keywords={keyword}'
            else:
                url = f'https://cafef.vn/tim-kiem/trang-{page_number}.chn?keywords={keyword}'

            print(f"[Đang xử lý trang]: {url}")
            success = crawl_page(url, writer)
            if not success:
                print(f"[Kết thúc tìm kiếm từ khóa '{keyword}' tại trang {page_number}]")
                break

            page_number += 1
            if page_number > 100:
                print(f"[Dừng lại sau 100 trang để tránh quá tải]")
                break

print("\n🎉 ĐÃ HOÀN THÀNH CRAWL DỮ LIỆU!")



=== BẮT ĐẦU TÌM KIẾM: petrolimex ===
[Đang xử lý trang]: https://cafef.vn/tim-kiem.chn?keywords=petrolimex
[✓] Đã lưu bài: Petrolimex(PLX) báo lãi sụt giảm 81%, đang có hơn tỷ USD đem gửi ngân hàng và “rót” vào trái phiếu
[✓] Đã lưu bài: ĐHCĐPetrolimex: Doanh thu mất 1.000 tỷ đồng sau biến động giá dầu, cơ hội mở rộng mạng lưới bán lẻ từ việc sáp nhập tỉnh, thành
[✓] Đã lưu bài: Tổng công ty Hóa dầuPetrolimexđặt kế hoạch lợi nhuận năm 2025 cao gấp đôi, cổ tức tối thiểu 12%
[✓] Đã lưu bài: PetrolimexAviation: 17 năm đồng hành phát triển cùng ngành hàng không, cùng đất nước
[✓] Đã lưu bài: Hết năm 2024, quỹ bình ổn xăng dầu còn hơn 6.000 tỷ đồng, một nửa nằm ởPetrolimex
[✓] Đã lưu bài: ĐHĐCĐ bất thườngPetrolimex: Thông qua việc sáp nhập TCT Dịch vụ Xăng dầuPetrolimexvào Tập đoàn
[✓] Đã lưu bài: Petrolimex, EVN cảnh báo người dân không ấn vào link lạ, không chuyển tiền cho các trang Facebook giả mạo
[✓] Đã lưu bài: PetrolimexAviation tái cung cấp nhiên liệu cho Azur Air sau gần 3 năm giá